<a href="https://colab.research.google.com/github/bonetrade/text-classifier/blob/main/text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

following this: https://www.section.io/engineering-education/classification-model-using-bert-and-tensorflow/

In [ ]:
!pip install tensorflow_text

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [41]:
df = pd.read_csv("tweets-d+h-v-s.csv")

In [42]:
df.head(5)

,text,category
0,@mercymarss_ @InADash I thought Latino was a m...,derogatory
1,"Near the end of 2021 now, with all the informa...",support
2,@MsRiceBite @GinaPhi94724925 So I’m guessing y...,derogatory
3,@dancingchefabby @nashbash2 Hey! Here's some m...,support
4,@paleomagick As far as I know it is a personal...,support


In [43]:
df['category'].value_counts()

derogatory    519
support       259
Name: category, dtype: int64

In [44]:
df_derogatory = df[df['category']=='derogatory']
df_support = df[df['category']=='support']

In [45]:
df_derogatory_downsampled = df_derogatory.sample(df_support.shape[0])

df_derogatory_downsampled.shape

(259, 2)

In [47]:
df_balanced = pd.concat([df_derogatory_downsampled, df_support])

In [40]:
#del df

In [48]:
df_balanced

,text,category
509,@votesamuelwill1 @Aenigma777 Any team using In...,derogatory
563,@MythinformedMKE I don’t understand why these ...,derogatory
1113,"Gimme Honda, Gimme Sony. So cheap and real pho...",derogatory
424,"Gimme Honda, Gimme Sony. So cheap and real pho...",derogatory
804,New program for Quebec's Indian and Eskimo pop...,derogatory
...,...,...
1126,@_you_WHAT @Superdillin Cool I was called a dy...,support
1127,"@ABDLgirl English &amp; pig Latin fluently, a ...",support
1130,@jjo431 @iheartmindy From someone who lives in...,support
1132,@cotto1 @FBI Orange skin really You are racist...,support


In [52]:
df_balanced['category'].value_counts()

derogatory    259
support       259
Name: category, dtype: int64

In [53]:
df_balanced['derogatory']=df_balanced['category'].apply(lambda x: 1 if x=='derogatory' else 0)
df_balanced.sample(5)

,text,category,derogatory
480,Indian is an incorrect term. Redskin is a slur...,support,0
441,"@WokeTemple ""Asians"" is pretty vast denominati...",support,0
686,This sunset labradorite is from Canada &amp; i...,derogatory,1
1073,@GayleKing cont. are guilty of racism. I was c...,support,0
969,@jaketapper I teach my k-8 students all the ti...,support,0


In [54]:
from sklearn.model_selection import train_test_split

In [56]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced['text'],df_balanced['derogatory'], stratify=df_balanced['derogatory'])

In [57]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [58]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [59]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)


In [60]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [61]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [62]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [77]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
13/13 [==============================] - 9s 654ms/step - loss: 0.6236 - accuracy: 0.6649 - precision: 0.6975 - recall: 0.5825
Epoch 2/100
13/13 [==============================] - 8s 643ms/step - loss: 0.5983 - accuracy: 0.6753 - precision: 0.6560 - recall: 0.7371
Epoch 3/100
13/13 [==============================] - 8s 647ms/step - loss: 0.6088 - accuracy: 0.6598 - precision: 0.7385 - recall: 0.4948
Epoch 4/100
13/13 [==============================] - 8s 645ms/step - loss: 0.6060 - accuracy: 0.6753 - precision: 0.6504 - recall: 0.7577
Epoch 5/100
13/13 [==============================] - 8s 635ms/step - loss: 0.6080 - accuracy: 0.6649 - precision: 0.7133 - recall: 0.5515
Epoch 6/100
13/13 [==============================] - 8s 634ms/step - loss: 0.6106 - accuracy: 0.6649 - precision: 0.6495 - recall: 0.7165
Epoch 7/100
13/13 [==============================] - 8s 636ms/step - loss: 0.6164 - accuracy: 0.6830 - precision: 0.7261 - recall: 0.5876
Epoch 8/100
13/13 [===============

In [78]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [79]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1])

In [108]:
# actual tweets
sample_dataset = [
 "We're barely halfway through the day and an Indian man flirting with me said thinking about being fucked by a HALF BREED like you is hot",
 "Antique Vintage Alaskan Eskimo Indian Wall Piece w/ Fur",
 "Manitoba History: The Red River Jig Around the Convention of Indian Title: The Metis and Half-Breed Dos a Dos",
 'My 7th grade history teacher would call me ‘redskin girl’ every time she saw me because she wanted everyone to know I was indian',
 "Actually let me retort if being American is all about Lying, Stealing, Destroying other peoples property then I don’t want to be your classification of an American I’d rather be called Native, Redskin, Indian or whatever else it is you Libtards call us without an audience",
 "I just want to make sure I understand - Inuit *used to be* included in the Indian act but are no longer, because the term Eskimo was excluded and the proper term Inuit was not subsequently included?",  
 "I understand Cree FN have been excluded from important rights as well.",
 "I am viewing This from a different perspective ,This is the most iconic scene in that movie, will fill the viewer with lot of emotion and pride for ancient Indian heritage. the creator of  this video ajmal sabu by name itself a half breed will always wish to malign this scene.",
 "It's a word you should never say to a native Indian but it's socially acceptable in a normal conversation (redskin).But the word has evolved into something with more than one connotation where context matters.",
 "eskimo eater of fish"

]

Closer the score gets to 1, the more likely that the tweet expresses 'hate'

In [112]:
model.predict(sample_dataset)

array([[0.33278838],
       [0.91821074],
       [0.62164605],
       [0.402493  ],
       [0.24394764],
       [0.4155691 ],
       [0.25358894],
       [0.49854347],
       [0.25175098],
       [0.9112105 ]], dtype=float32)

In [107]:
tf.keras.models.save_model(model, "model")

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [111]:
!zip -r hatedetectionmodel.zip model/

  adding: model/ (stored 0%)
  adding: model/keras_metadata.pb (deflated 84%)
  adding: model/assets/ (stored 0%)
  adding: model/assets/vocab.txt (deflated 53%)
  adding: model/variables/ (stored 0%)
  adding: model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: model/variables/variables.index (deflated 79%)
  adding: model/saved_model.pb (deflated 92%)


so i need to make a new notebook that will load the model, and apply it against a csv of tweets loaded up as a dataframe. see also https://stackoverflow.com/questions/62832159/how-to-save-full-tensorflow-model-from-google-colab for saving/loading a model